In [1]:
import matlab.engine
import pandas
from keras.models import load_model
import random

In [2]:
#przesunięcie dryfu czujnika wartości <-1 do 1>
#w tym 0 oznacza brak dryfu
SDrift = -0.8
#parametr wzmocnienia zwarcia zęba/ uszkodzenia zęba przekładni wartosci <-2 do 0>
# przy czym 0 to brak uszkodzenia zęba
SGain = 0
#parametr shaft wear -> zużycie wału parametr [0, -1] 
#0-> brak uszkodzenia
#-1 ->zużycie walu
ShaftWear = 0

In [3]:
eng = matlab.engine.start_matlab()
correct = eng.SimulateTransmissionCasing(SDrift,SGain,ShaftWear)

In [4]:
eng.exit()
simulation = pandas.read_parquet("SimulationData.parquet")

In [5]:
simulation

,vibration,tacho1,tacho2
0,-0.800000,1.0,1.0
1,-0.800000,1.0,1.0
2,-0.800000,1.0,1.0
3,-0.800000,1.0,1.0
4,-0.800000,1.0,1.0
...,...,...,...
395,0.482954,0.0,0.0
396,-0.624341,0.0,0.0
397,-1.428926,0.0,0.0
398,-1.349077,0.0,0.0


In [6]:
simulation = simulation.iloc[100:,:]

In [7]:
simulation.reset_index(drop=True)

,vibration,tacho1,tacho2
0,-0.632228,0.0,0.0
1,-1.351946,0.0,0.0
2,-1.426686,0.0,0.0
3,-0.618158,0.0,0.0
4,0.488226,0.0,0.0
...,...,...,...
295,0.482954,0.0,0.0
296,-0.624341,0.0,0.0
297,-1.428926,0.0,0.0
298,-1.349077,0.0,0.0


In [8]:
model = load_model('PredictFaultData')

In [9]:
X_simulation = simulation.values

In [10]:
n_signals, n_steps, n_length = 3, 20, 15
X_simulation = X_simulation.reshape((1, n_steps, n_length, n_signals))

In [11]:
X_simulation

array([[[[-0.63222814,  0.        ,  0.        ],
         [-1.35194628,  0.        ,  0.        ],
         [-1.42668598,  0.        ,  0.        ],
         [-0.61815778,  0.        ,  0.        ],
         [ 0.48822567,  0.        ,  0.        ],
         [ 1.01488421,  0.        ,  0.        ],
         [ 0.59277408,  0.        ,  0.        ],
         [-0.3041332 ,  0.        ,  0.        ],
         [-0.79771761,  0.        ,  0.        ],
         [-0.41460483,  0.        ,  0.        ],
         [ 0.47814046,  0.        ,  0.        ],
         [ 1.01083886,  0.        ,  0.        ],
         [ 0.61215576,  0.        ,  0.        ],
         [-0.46273747,  0.        ,  0.        ],
         [-1.36216561,  0.        ,  0.        ]],

        [[-1.4149876 ,  0.        ,  0.        ],
         [-0.74100656,  0.        ,  0.        ],
         [-0.13420017,  0.        ,  0.        ],
         [-0.32631536,  0.        ,  0.        ],
         [-1.30765478,  0.        ,  0.        ]

In [12]:
predictions = model.predict(X_simulation)

In [13]:
model_SensorDrift = load_model('PredictFaultData_SensorDrift');
model_ShaftWear = load_model('PredictFaultData_ShaftWear');
model_ToothFault = load_model('PredictFaultData_ToothFault');

In [14]:
prediction_SensorDrift = model_SensorDrift.predict(X_simulation)
prediction_ShaftWear = model_ShaftWear.predict(X_simulation)
prediction_ToothFault = model_ToothFault.predict(X_simulation)

In [15]:
prediction_SensorDrift

array([[0.59930885]], dtype=float32)

In [16]:
prediction_ShaftWear

array([[0.08464664]], dtype=float32)

In [17]:
prediction_ToothFault

array([[0.00020415]], dtype=float32)

In [18]:
list_prediction_SensorDrift = prediction_SensorDrift.tolist()
list_prediction_ShaftWear = prediction_ShaftWear.tolist()
list_prediction_ToothFault = prediction_ToothFault.tolist()

In [19]:
list_prediction_ToothFault[0][0]

0.0002041459083557129

In [20]:
threshpreds = [(list_prediction_SensorDrift[0][0] > 0.4) * 1,(list_prediction_ShaftWear[0][0] > 0.4) * 1, (list_prediction_ToothFault[0][0] > 0.4) * 1]

In [21]:
threshpreds

[1, 0, 0]